<a href="https://colab.research.google.com/github/Midmost/AI/blob/master/EDA_gangnam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install --upgrade pandas

## Basic information

In [ ]:
import pandas as pd
import numpy as np
from pandas.io.formats.info import DataFrameInfo

# excel path
xlsx_file_path = "/content/drive/MyDrive/DATA/pdf2xlsx/editXL/X_series/X_강남구_예산서.xlsx"

# pure excel to pure dataframe and information with a visual-table
df = pd.read_excel(xlsx_file_path)
DataFrameInfo
df

In [ ]:
# drop the 0,1,2 rows because of starting page
df = pd.read_excel(xlsx_file_path, header = None)

# gu var
gu_path = xlsx_file_path.split('_')
gu_name = gu_path[2]

# Add a new column to the first index of the dataframe
df.insert(0, '구', gu_name)
header_all = "구,부서,정책,단위,세부사업,편성목,편성운영비,편성세부운영비,편성세부내역비용,예산액,전년도예산액,비교증감"
header = header_all.split(",")
df.columns = header

* ***하지만 만약 xlsx 파일 혹은 pdf 파일을 확인하고 내용이 다르다면 그에 맞춰 header를 수정해야 할 수 있음*** 

* 헤더의 명칭은 각 구의 계층구조에 정확히 맞춰서 수정하였음. 

In [ ]:
# columns_to_fill = header[:5] 이지만 혹시나 헤더명이 변경될 수 있으므로 아래와 같이 사용한다. 
columns_to_fill = ['부서', '정책', '단위', '세부사업', '편성목', '편성운영비']

for col in columns_to_fill:
    df[col] = df[col].fillna(method='ffill')

# 데이터목수 대표님이 주신 양식에 맞추기 
df['편성세부운영비'] = df['편성세부운영비'].astype(str).replace('nan', '')
df['편성세부내역비용'] = df['편성세부내역비용'].astype(str).replace('nan', '')
df['편성세부운영비'] = df['편성세부운영비'] + df['편성세부내역비용']
df['편성세부운영비']
df = df.drop(columns=['편성세부내역비용'])
df

In [ ]:

# # TEST '01'일 때만 확인해보기 


# # all values of column index 6 should be changed to broadcasting_expenses,
# # 편성운영비 == briadcasting_expenses
# # 1. if b_e is in the index 7 column, extract each value and save in the new column 
# briadcasting_expenses =  ["{:02d}".format(i) for i in range(1, 100)]
# b_e = briadcasting_expenses

# type(b_e) #str ['01','02','03',...'99']

# # Create a boolean mask to identify rows where the value in the '편성세부운영비' column starts with '01'
# mask = df['편성세부운영비'].str.startswith('01')

# # Use the mask to set the values in the '편성운영비' column
# df.loc[mask, '편성운영비'] = df.loc[mask, '편성세부운영비']

# # df = df[df['편성운영비'] != '본예산']
# df

In [ ]:
# Create a list of values to check
values = ["{:02d}".format(i) for i in range(1, 11)]

# Create a boolean mask to identify rows where the value in the '편성세부운영비' column starts with any value in the list
mask = df['편성세부운영비'].str.startswith(tuple(values))

# Use the mask to set the values in the '편성운영비' column
df.loc[mask, '편성운영비'] = df.loc[mask, '편성세부운영비']

# Input Null all rows in the '편성운영비' column where the value is '본예산'
# Set all rows in the '편성운영비' column where the value is '본예산' to NaN
df.loc[df['편성운영비'] == '본예산', '편성운영비'] = np.nan

# Create a boolean mask to identify rows where the value in the '편성세부운영비' column contains any value in the list
mask = df['편성세부운영비'].str.contains('|'.join(values))

# Use the mask to set the values in those rows to NaN
df.loc[mask, '편성세부운영비'] = np.nan

df

In [ ]:
mask

In [ ]:
# Export the DataFrame to an Excel file
df.to_excel('output_gangnam.xlsx', index=False)